In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

## Загрузка данных:

In [ ]:
dataset = pd.read_excel(r'C:\Users\kakorin_is\Desktop\Аналитик\выгрузка на 15 февраля.xlsx', index_col=None)

earliers_doned = pd.read_excel(r'C:\Users\kakorin_is\Desktop\Свод\Перечень учтенных ранее ПА (с учетом 3 квартала 2022 года).xlsx', index_col=None)

activities = pd.read_excel(r'C:\Users\kakorin_is\Desktop\JupiterNotebook_files\активности нст на 15 февр.xlsx', index_col=None)


In [ ]:
dataset.shape

(5489, 59)

In [ ]:
# cмотрим список колонок стандартной выгрузки из БД (не все будут нужны)
dataset.columns

Index(['Номер по SIEBEL', 'РОО', 'Договор', 'Заемщик ФЛ', 'Заемщик КМБ', 'Тип',
       'Стадия', 'Сумма заявленных требований',
       'Номер дела в суде (1-ая инстанция)', 'Суд', 'Передача обращений в суд',
       'Критерий нестандартности', 'Дата судебного решения',
       'Дата последнего судебного заседания',
       'Результат последнего судебного заседания', 'Тип последней активности',
       'Дата начала последней активности', 'Сумма взыскания по решению суда',
       'Комментарии', 'Дата создания дела', 'Результат', 'Дата закрытия',
       'Статус', 'Нормативный срок получения ИД', 'Дело на контроле',
       'Сайт суда', 'Общая сумма ПЗ', 'Общая сумма ПЗ по ОД',
       'Сумма просроченных процентов', 'Сумма просроченных комиссий',
       'Передано на аутсорсинг/внешние консультанты',
       'Организация/ФИО привлеченного ВК', 'Куратор/Координатор по делу',
       'Пени', 'Продукт', 'Создатель', 'Передано в ЮУ/ЮСТП', 'Дата передачи',
       'ЦС подсудности', 'ТП подсудности', 'На

In [ ]:
# так он выглядит (для примера вывожу обезличенные столбцы)
dataset.loc[:, ['Номер по SIEBEL','РОО','Тип','Дата создания дела','Дата закрытия', ]]

,Номер по SIEBEL,РОО,Тип,Дата создания дела,Дата закрытия
0,1-2946161483,Нальчик,Банкротство ФЛ РБ,2022-10-21 08:55:46,2022-12-02
1,1-2922685695,Нальчик,Взыскание задолженности ПА РБ,2022-10-11 11:12:52,NaT
2,1-2777993227,Нальчик,Банкротство ФЛ РБ,2022-08-24 15:11:39,2022-10-05
3,1-2829411327,Нальчик,Банкротство ФЛ РБ,2022-09-09 15:08:09,2022-11-17
4,1-2562329796,Нальчик,Банкротство ФЛ РБ,2022-06-03 15:10:32,2022-08-24
...,...,...,...,...,...
5484,1-1533535923,Краснодар,Банкротство ФЛ РБ,2021-05-24 14:14:21,2021-08-13
5485,1-1643646792,Краснодар,Банкротство ФЛ РБ,2021-06-29 13:32:18,2022-01-10
5486,1-1640798290,Краснодар,Взыскание задолженности ПА МБ,2021-06-28 11:50:58,2021-07-06
5487,1-1619257247,Краснодар,Банкротство ФЛ РБ,2021-06-16 15:02:39,2022-01-10


In [ ]:
data = dataset.copy()

In [ ]:
# Глобальные переменные - потом прописать сюда

# курс доллара и евро
# дата начала квартала
# дата конца квартала
# названия файлов загрузки

## Описание алгоритма:

- Преобразовать поле Дата создания дела в поле с только с датой (убрать время).
<br>
- Удалить дела, закрытые до отчетного квартала и завершенные в этом квартале результатами Возврат, Отказано, а также отмененые дела.
<br>
- Подтянуть по номеру по сибель и договору ранее учтенные дела, чтобы потом исключить их из СМОТ
<br>
- Подтянуть по номеру по сибель активность НСТ из выгрузки нестандартных активностей, чтобы разметить стандартные и нест. дела (разные нормативные сроки у них)
<br>
- Коррекция сумм: <br>
    а) убрать фразу Руб. из Заявл. требований и Суммы взысканного по решению суда.<br>
    б) убрать символы доллара и евро из Заявл. требований и при этом перевести суммы из валюты в рубли.<br>
    в) по полю Валюта найти валютные кд и преобразовать колонки Сумма ОД и % по ним,
    
 <br>
- Коррекция колонки Тип - в отдельном столбце Тип сокр преобразуем направления бизнеса.
- Создание колонки Дата конца (имеется ввиду дата конца квартала - будем из нее вычитать).
- Создание колонок: Кол-во дней, Коэф,  Факт, План.
<br>
- Заполнеие новых колонок:<br>
    а) Кол-во дней - по активным делам вычитаем из даты конца квартала дату создания, а по закрытым из даты закрытия - дату создания<br>
    б) Коэф - исходя из значений поля Стандарт/нестандарт и поля Кол-во дней в проставляется соотв. коэф. с учетом утвержденных сроков работы с делами.<br>
    в) Факт - заполняется, исходя из проставленного коэф., путем умножения к. на сумму ОД и % по делам о взыскании либо, если дело банкротное, путем умножения к. на Заявл. требования.<br>
    г) План - заполняется по аналогии с Фактом.

## Правим поле Дата создания дела:

In [ ]:
# Функция преобразования даты
def fix_date(some_date):
    some_date = some_date.date()
    
    return some_date

# Специально делаю отдельный столбец к корректной формой даты, чтобы можно было потом сравнить
data['Дата создания'] = data['Дата создания дела'].apply(lambda x: fix_date(x))

In [ ]:
# Проверим корректно ли произошли преобразования
data.loc[:, ['Дата создания дела', 'Дата создания']]

,Дата создания дела,Дата создания
0,2022-10-21 08:55:46,2022-10-21
1,2022-10-11 11:12:52,2022-10-11
2,2022-08-24 15:11:39,2022-08-24
3,2022-09-09 15:08:09,2022-09-09
4,2022-06-03 15:10:32,2022-06-03
...,...,...
5484,2021-05-24 14:14:21,2021-05-24
5485,2021-06-29 13:32:18,2021-06-29
5486,2021-06-28 11:50:58,2021-06-28
5487,2021-06-16 15:02:39,2021-06-16


## Удаляем дела:

Поскольку выгрузка из БД содержит все вообще дела за все время, то нужно оставить только активные дела и завершенные в этом квартале.

In [ ]:
# правим тип данных
data['Дата закрытия'] = data['Дата закрытия'].apply(lambda x: fix_date(x))

In [ ]:
to_del_old = data.loc[(data['Статус'] == 'Завершено') & (datetime(2023, 1, 1, 0, 0, 0, 1).date() > data['Дата закрытия'])]
to_del_old.index

Int64Index([   0,    2,    3,    4,    6,    8,    9,   10,   11,   12,
            ...
            5479, 5480, 5481, 5482, 5483, 5484, 5485, 5486, 5487, 5488],
           dtype='int64', length=4239)

In [ ]:
# удаляю все завершенное до 01.01.2023
data = data.drop (index=to_del_old.index)
data.shape

(1250, 60)

In [ ]:
data['Статус'].value_counts()

В работе             818
Завершено            373
Отменено              49
Приостановлено         5
Частично передано      5
Name: Статус, dtype: int64

In [ ]:
data = data.loc[data['Статус'] != 'Отменено']
data.shape

(1201, 60)

In [ ]:
# Смотрим какие Результаты имеются в оставленных делах
data.loc[data['Статус'] == 'Завершено']['Результат'].value_counts()

Включено в РТК                300
ИД передан в УПК               36
Возврат в досудебное пр-во     14
Включено за РТК                14
Получен судебный акт            3
Создано ошибочно                3
Отказано во вкл в РТК           2
Name: Результат, dtype: int64

In [ ]:
# Из выгрузки нужно удалить Возвращенное, Ошибочное и Отказные дела

to_del_2 = data.loc[(data['Статус'] == 'Завершено') & ((data['Результат'] == 'Возврат в досудебное пр-во') | \
                                                       (data['Результат'] == 'Создано ошибочно') | \
                                                       (data['Результат'] == 'Отказано во вкл в РТК'))]
to_del_2 ['Результат'].value_counts()

Возврат в досудебное пр-во    14
Создано ошибочно               3
Отказано во вкл в РТК          2
Name: Результат, dtype: int64

In [ ]:
data = data.drop (index=to_del_2 .index)
data.shape

(1182, 60)

## Подтягиваем ранее учтенные:

In [ ]:
# смотрим какие колонки в таблице и ее размер
earliers_doned.columns, earliers_doned.shape

(Index(['Номер по SIEBEL', 'Договор', 'Квартал учета в факте'], dtype='object'),
 (25275, 3))

In [ ]:
earliers_doned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25275 entries, 0 to 25274
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Номер по SIEBEL        25275 non-null  object
 1   Договор                14342 non-null  object
 2   Квартал учета в факте  25275 non-null  object
dtypes: object(3)
memory usage: 592.5+ KB


В таблице earliers_doned содержится список кд и номеров по Сибель дел, которые ранее учитывались в СМОТ и их нужно исключать.<br>

Также видим, что в некоторых случаях отдельно зафиксированы номера кд по учтенным делам, а отдельно номера по Сибель - будем учитывать это при подтягивании значений

In [ ]:
# PART 1
# присоединяю учтенные по номеру по сибель
data = pd.merge(data, earliers_doned,  on ='Номер по SIEBEL', how ='left')

# переписываю значения в колонке Учтен
data['Учтен'] = data['Квартал учета в факте']

# после того как переписал, удаляю смерженные столбцы - из-за наличия в исходной выгрузки колонки Договор, 
# она становится Договор_x, а присоединный столбец из выгрузки учтенных - Договор_y
data.drop(columns=['Квартал учета в факте', 'Договор_y'], inplace=True)

# и возвращаю корректное название колонке Договор
data.rename(columns={'Договор_x' : 'Договор'}, inplace=True)

# смотрим что получилось 
data['Учтен'].value_counts()

основное дело у ЮД    1
3 квартал 2022        1
3 квартал 2019        1
Name: Учтен, dtype: int64

In [ ]:
#PART 2
# осталось присоединить по договору, чтобы это сделать удалю в исходном списке учтенных пустые строки в колонке Договор
nan_list_1 = earliers_doned[pd.isnull(earliers_doned['Договор'])].index
earliers_doned_2 = earliers_doned.drop (index=nan_list_1)

#присоеднияю по Договору 
data = pd.merge(data, earliers_doned_2,  on ='Договор', how ='left')

data['Квартал учета в факте'].value_counts()

3 квартал 2021    5
3 квартал 2019    2
4 квартал 2020    2
2 квартал 2019    1
4 квартал 2019    1
Name: Квартал учета в факте, dtype: int64

Первая смерженная часть инф. об учете ПА была выше перенесена в поле Учет и после второго мержа у нас снова появилась колонка Квартал учета в факте, откуда будем переносить данные в колонку Учет.

__Примечание:__ заполнение полей через цикл и индексы здесь использую для разнообразия, в дальнейшем внизу буду работать с методом .apply() 

In [ ]:
# делаю список нанов чтобы оставить только строки с заполненным значением учета в Факте, чтобы взять индексы строк и переписать
nan_list_2 = data[pd.isnull(data['Квартал учета в факте'])].index                                                  # в цикле
hidden_data = data.drop(index=nan_list_2)
hidden_data.index

Int64Index([149, 275, 304, 305, 543, 552, 809, 822, 823, 1158, 1165], dtype='int64')

In [ ]:
for indx in hidden_data.index:
    data.loc[indx, 'Учтен'] = data.loc[indx, 'Квартал учета в факте']

In [ ]:
# проверяем
data['Учтен'].value_counts()

3 квартал 2021        5
3 квартал 2019        2
4 квартал 2020        2
основное дело у ЮД    1
2 квартал 2019        1
3 квартал 2022        1
4 квартал 2019        1
Name: Учтен, dtype: int64

In [ ]:
# смерженные столбцы не нужны - удаляем
data.drop(columns=['Квартал учета в факте', 'Номер по SIEBEL_y'], inplace=True)

# и возвращаем корректные названия колонкам 
data.rename(columns={'Номер по SIEBEL_x' : 'Номер по SIEBEL'}, inplace=True)

## Подтягиваем активности НСТ:

In [ ]:
activities.columns

Index(['Мероприятие', 'Индикатор', 'Тип', 'Тип активности Outbound', 'Статус',
       'Начало', 'Фактическое окончание', 'Результат', 'Соотношение к ПЗ',
       'Завершена с просрочкой', 'Клиент (ФЛ)', 'КМБ', 'Комментарии',
       '№ договора', 'Судебное дело', 'Адрес',
       'Дней просрочки на момент создания', 'Ответственный по клиенту',
       'Центр сбора', 'Номер дела о банкротстве', 'Ответственный',
       'Дата создания', 'Дата передачи', 'Общий комментарий', 'Описание',
       'Иная судебная работа', 'Дата публичных торгов (принятие на баланс)'],
      dtype='object')

In [ ]:
# в выгрузке активностей нужно всего два столбца - возьму их и переименую колонки в корректные названия,
# т.к. мержить надо будет по номеру по Сибель. Здесь в колонке "Судебное дело" содержится Номер по Сибель

activities = activities.loc[:, ['Судебное дело', 'Статус']]
activities.rename(columns={'Судебное дело' : 'Номер по SIEBEL', 'Статус': 'Статус активности нст'}, inplace=True)

In [ ]:
activities.head(2)

,Номер по SIEBEL,Статус активности нст
0,1-2237097491,Запланировано
1,1-531879843,Запланировано


In [ ]:
# переименую значения в 1 вид (так проще перезаписывать потом)
activities['Статус активности нст'].replace("Запланировано", "Завершено", inplace=True) 
activities['Статус активности нст'].value_counts()

# отмененные не будем метить как нестандртные - они ошибочны

Завершено    265
Отменено      15
Name: Статус активности нст, dtype: int64

In [ ]:
# отмененные отсеку, посколько по некоторым делам имеется несколько активностей НСТ и, если в таком деле будет отмененная, то 
# она подтянется и мы не поймем, что дело на самом деле нестандартной, т.к. может быть не учтена вторая корректная активность НСТ

activities = activities.loc[activities['Статус активности нст'] == 'Завершено']
activities['Статус активности нст'].value_counts()

Завершено    265
Name: Статус активности нст, dtype: int64

In [ ]:
data = pd.merge(data, activities,  on ='Номер по SIEBEL', how ='left')
data['Статус активности нст'].value_counts()

Завершено    86
Name: Статус активности нст, dtype: int64

In [ ]:
# исходя из наличия корректных активностей, заполняем поле СТ/НСТ
data['СТ/НСТ'] = np.where(data['Статус активности нст'] == 'Завершено' , 'НСТ', 'СТ')
data['СТ/НСТ'].value_counts()

СТ     1119
НСТ      86
Name: СТ/НСТ, dtype: int64

## Корректируем суммы:

In [ ]:
# у наших колонок тип данных объект, но если брать отдельные значения, то будет строковые данные:

data.loc[:,['Сумма заявленных требований', 'Сумма взыскания по решению суда']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1204
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Сумма заявленных требований      1154 non-null   object
 1   Сумма взыскания по решению суда  1205 non-null   object
dtypes: object(2)
memory usage: 28.2+ KB


In [ ]:
list_to_replace = ['Сумма заявленных требований', 'Сумма взыскания по решению суда']


def replace_and_fix_values(x):
    if type(x) == str: # обхожу ошибку типа данных, когда встречается ноль
        
        # обрабатываем Руб.
        x_fixed = x.replace("Руб.", "")
        
        # приводим в корректный вид для преобразования в числовой формат
        x_fixed = x_fixed.replace(" ", "")    # пробелы между цифрами нужно также убрать
        x_fixed = x_fixed.replace(",", ".")   # в питоне у флоата точка вместо запятой
        
        # сразу убираю символ доллара и перевожу доллары в рубли
        if '$' in x_fixed:
            x_fixed = x_fixed.replace("$", "")
            x_fixed = str(float(x_fixed) * 70)  # 70 берем за текущий курс доллара - в перспективе переписать в глоб. переменную
        
        # аналогичное для евро
        if '€' in x_fixed:
            x_fixed = x_fixed.replace("€", "")
            x_fixed = str(float(x_fixed) * 70)  # на начало 2023 г. курсы валют примерно одинаковые
        
        return x_fixed
    return x

for i in list_to_replace:
    data[i] = data[i].apply(lambda x: replace_and_fix_values(x))

In [ ]:
# смотрим результат
data.loc[:, list_to_replace].head(-4)

,Сумма заявленных требований,Сумма взыскания по решению суда
0,120106.50,0
1,222163.15,0
2,384892.89,0
3,59007.70,1970.23
4,51214.36,51214.36
...,...,...
1196,288021.48,288021.48
1197,396364.24,0
1198,346356.65,0
1199,555649.17,555649.17


In [ ]:
# переписываю тип данных
data = data.astype({'Сумма заявленных требований': np.float64, 'Сумма взыскания по решению суда': np.float64})

data.loc[:,['Сумма заявленных требований', 'Сумма взыскания по решению суда']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1204
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Сумма заявленных требований      1154 non-null   float64
 1   Сумма взыскания по решению суда  1205 non-null   float64
dtypes: float64(2)
memory usage: 28.2 KB


In [ ]:
data.loc[:,['Сумма заявленных требований', 'Сумма взыскания по решению суда']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1204
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Сумма заявленных требований      1154 non-null   float64
 1   Сумма взыскания по решению суда  1205 non-null   float64
dtypes: float64(2)
memory usage: 28.2 KB


In [ ]:
# суммы отображаются корректно
data.loc[:, list_to_replace].head(3)

,Сумма заявленных требований,Сумма взыскания по решению суда
0,120106.50,0.0
1,222163.15,0.0
2,384892.89,0.0


__Смотрим что в поле Валюта__:

In [ ]:
data['Валюта заявленных требований'].value_counts()

RUB    1152
USD       2
Name: Валюта заявленных требований, dtype: int64

In [ ]:
data.loc[data['Валюта заявленных требований'] == 'USD', ['Общая сумма ПЗ по ОД', 'Сумма просроченных процентов']]

,Общая сумма ПЗ по ОД,Сумма просроченных процентов
11,2631.79,0.0
12,2631.79,0.0


__Меняем валютные значения у колонок:__

In [ ]:
data.loc[data['Валюта заявленных требований']=='USD', 'Общая сумма ПЗ по ОД'] *= 70
data.loc[data['Валюта заявленных требований']=='USD', 'Сумма просроченных процентов'] *= 70

data.loc[data['Валюта заявленных требований']=='EUR', 'Общая сумма ПЗ по ОД'] *= 70
data.loc[data['Валюта заявленных требований']=='EUR', 'Сумма просроченных процентов'] *= 70

# сравниваем как прошел перевод - строкой выше цифры до преобразования
data.loc[data['Валюта заявленных требований'] == 'USD', ['Общая сумма ПЗ по ОД', 'Сумма просроченных процентов']]

,Общая сумма ПЗ по ОД,Сумма просроченных процентов
11,184225.3,0.0
12,184225.3,0.0


Тип сокр и РОО

##  Переписываем поле Тип:

Суть в том, что Банкроты ФЛ могут быть МБ или РБ, но Банкротов ФЛ МБ нужно учесть как МБ

In [ ]:
data['Тип сокр'] = data['Тип']

In [ ]:
data['Тип сокр'] = data['Тип сокр'].convert_dtypes()

In [ ]:
data['Тип сокр'].value_counts()

Банкротство ФЛ РБ                831
Взыскание задолженности ПА РБ    312
Взыскание задолженности ПА МБ     43
Банкротство ЮЛ МБ                 14
Банкротство ФЛ МБ                  5
Name: Тип сокр, dtype: Int64

In [ ]:
data['Тип сокр'] = np.where((data['Тип сокр'] == 'Банкротство ФЛ РБ'), 'Банкротство РБ', data['Тип сокр'])
data['Тип сокр'] = np.where((data['Тип сокр'] == 'Банкротство ЮЛ МБ'), 'Банкротство МБ', data['Тип сокр'])    
data['Тип сокр'] = np.where((data['Тип сокр'] == 'Банкротство ФЛ МБ'), 'Банкротство МБ', data['Тип сокр'])    
data['Тип сокр'] = np.where((data['Тип сокр'] == 'Взыскание задолженности ПА РБ'), 'Взыскание РБ', data['Тип сокр'])
data['Тип сокр'] = np.where((data['Тип сокр'] == 'Взыскание задолженности ПА МБ'), 'Взыскание МБ', data['Тип сокр'])


In [ ]:
# смотрим результат
data.groupby(['Тип сокр', 'Тип']).agg({'Номер по SIEBEL': 'count'})

# все корректно

Номер по SIEBEL
Тип сокр       Тип                                           
Банкротство МБ Банкротство ФЛ МБ                            5
               Банкротство ЮЛ МБ                           14
Банкротство РБ Банкротство ФЛ РБ                          831
Взыскание МБ   Взыскание задолженности ПА МБ               43
Взыскание РБ   Взыскание задолженности ПА РБ              312

## Создаем колонку дата конца:

In [ ]:
data['Дата конца'] = datetime(2023, 3, 31, 23, 59, 0, 1).date()
data['Дата конца'].value_counts()

2023-03-31    1205
Name: Дата конца, dtype: int64

In [ ]:
data.loc[:, ['Дата конца', ]]

,Дата конца
0,2023-03-31
1,2023-03-31
2,2023-03-31
3,2023-03-31
4,2023-03-31
...,...
1200,2023-03-31
1201,2023-03-31
1202,2023-03-31
1203,2023-03-31


__Добавляю колонки, значения по умолчанию будут None, чтобы потом можно было себя перепроверить__

In [ ]:
columns_to_add = ['Кол-во дней', 'Коэф', 'Факт', 'План']

for i in columns_to_add:
    data[i] = None

# Заполняем колонки:

__Колонка Кол-во дней:__

Здесь есть особенность по делам созданным либо завершенным в период с 15 дек по 19 янв - по ним срок увеличен на 35 дней. Это нужно учитывать

In [ ]:
# промежуточный вариант для понимания что будет выводиться
d = data.loc[data.index == 5]['Дата конца'] - data.loc[data.index == 5]['Дата создания']
d

5   171 days
dtype: timedelta64[ns]

In [ ]:
# так будем забирать только число
d.dt.days

5    171
dtype: int64

In [ ]:
# ф-ция расчета кол-ва дней  - по завершенным вычитается Дата закрытия из Даты создания, а по активным из Даты конца.

def count_num_days(row):
    if row['Статус'] == 'Завершено':
        if (row['Дата создания'] >= datetime(2022, 12, 15, 0, 0, 0, 1).date()) and \
        row['Дата создания'] <= datetime(2023, 1, 19, 0, 0, 0, 1).date():
            date_x = row['Дата закрытия'] - row['Дата создания'] - timedelta(days=35)
            return date_x.days
        
        if (row['Дата закрытия'] >= datetime(2022, 12, 31, 23, 59, 0, 1).date()) and \
        (row['Дата закрытия'] <= datetime(2023, 1, 19, 0, 0, 0, 1).date()):
            date_x = row['Дата закрытия'] - row['Дата создания'] - timedelta(days=35)
            return date_x.days
        
        date_x = row['Дата закрытия'] - row['Дата создания']
        return date_x.days
    
    # пошел блок по не завершенным делам
    else:
        if (row['Дата создания'] >= datetime(2022, 12, 15, 0, 0, 0, 1).date()) and \
        (row['Дата создания'] <= datetime(2023, 1, 19, 0, 0, 0, 1).date()):
            date_x = row['Дата конца'] - row['Дата создания'] - timedelta(days=35)
            return date_x.days
        
        # для всех остальных случаев
        date_x = row['Дата конца'] - row['Дата создания']
        return date_x.days
        
data['Кол-во дней'] = data.apply(count_num_days, axis=1)

In [ ]:
#  РАБОТАЕТ, ВЫКЛЮЧИЛ ЧТОБЫ НАПИСАТЬ опмтимальнее - см. выше

# indx_closed_deals = data.loc[data['Статус'] == 'Завершено'].index
# indx_active_deals = data.loc[data['Статус'] != 'Завершено'].index

# for indx in indx_closed_deals:
#     data.loc[indx, 'Кол-во дней'] = data.loc[indx, 'Дата закрытия'] - data.loc[indx, 'Дата создания']
    
# for indx in indx_active_deals:
#     data.loc[indx, 'Кол-во дней'] = data.loc[indx, 'Дата конца'] - data.loc[indx, 'Дата создания']
    
# # перевожу в числовой формат, чтобы можно было потом сравнивать
# data['Кол-во дней'] = data['Кол-во дней'].apply(lambda x: x.days)

__Сокращу кол-во колонок для возможности вывода обезличенной инф. с нужными для расчета коэффициента колонками:__

In [ ]:
cols_list = ['Номер по SIEBEL', 'РОО', 'Статус', 'Сумма заявленных требований','Сумма взыскания по решению суда',\
             'Дата создания', 'Дата конца','Комментарии','Дата закрытия','Учтен','Кол-во дней', 'Коэф','СТ/НСТ', \
             'Статус активности нст','Тип сокр','Общая сумма ПЗ по ОД','Сумма просроченных процентов','Факт', 'План']

data_cuted = data.loc[:, cols_list]
data_cuted.head(3)

,Номер по SIEBEL,РОО,Статус,Сумма заявленных требований,Сумма взыскания по решению суда,Дата создания,Дата конца,Комментарии,Дата закрытия,Учтен,Кол-во дней,Коэф,СТ/НСТ,Статус активности нст,Тип сокр,Общая сумма ПЗ по ОД,Сумма просроченных процентов,Факт,План
0,1-2922685695,Нальчик,В работе,120106.50,0.0,2022-10-11,2023-03-31,Иск.Наследники. Иск направлен в суд - 19.10.20...,NaT,NaN,171,None,СТ,NaN,Взыскание РБ,59281.32,7139.14,None,None
1,1-2566500297,Нальчик,В работе,222163.15,0.0,2022-06-06,2023-03-31,Иск. Наследники. Исковое заявление напр. в суд...,NaT,NaN,298,None,СТ,NaN,Взыскание РБ,175467.50,46695.65,None,None
2,1-2566858573,Нальчик,В работе,384892.89,0.0,2022-06-06,2023-03-31,Иск.Наследники. Исковое заявление напр. в суд ...,NaT,NaN,298,None,СТ,NaN,Взыскание РБ,315937.21,47946.16,None,None


In [ ]:
# выведу типы дел, чтобы под них написать ф-цию расчета коэффициента
data_cuted['Тип сокр'].value_counts()

Банкротство РБ    831
Взыскание РБ      312
Взыскание МБ       43
Банкротство МБ     19
Name: Тип сокр, dtype: int64

In [ ]:
# Коэф. одинаковые, но у стандартных дел свои сроки, у нест. - свои.
def get_coef(row):
    if row['Статус'] == 'Завершено':
        if row['СТ/НСТ'] == 'СТ':                     # Блок Завершенные СТ
            if row['Тип сокр'] == 'Банкротство РБ':
                if row['Кол-во дней'] <= 60:
                    return 1.5
                if (row['Кол-во дней'] > 60) and (row['Кол-во дней'] <= 100):
                    return 1
                if (row['Кол-во дней'] > 100) and (row['Кол-во дней'] <= 220):
                    return 0.3
                if row['Кол-во дней'] > 220:
                    return 0.7
                
            if row['Тип сокр'] == 'Банкротство МБ':
                if row['Кол-во дней'] <= 80:
                    return 1.5
                if (row['Кол-во дней'] > 80) and (row['Кол-во дней'] <= 170):
                    return 1
                if (row['Кол-во дней'] > 170) and (row['Кол-во дней'] <= 290):
                    return 0.3
                if row['Кол-во дней'] > 290:
                    return 0.7
                
            if row['Тип сокр'] == 'Взыскание МБ':
                if row['Кол-во дней'] <= 160:
                    return 1.5
                if (row['Кол-во дней'] > 160) and (row['Кол-во дней'] <= 180):
                    return 1
                if (row['Кол-во дней'] > 180) and (row['Кол-во дней'] <= 300):
                    return 0.3
                if row['Кол-во дней'] > 300:
                    return 0.7
                
            if row['Тип сокр'] == 'Взыскание РБ':
                if row['Кол-во дней'] <= 160:
                    return 1.5
                if (row['Кол-во дней'] > 160) and (row['Кол-во дней'] <= 170):
                    return 1
                if (row['Кол-во дней'] > 170) and (row['Кол-во дней'] <= 290):
                    return 0.3
                if row['Кол-во дней'] > 290:
                    return 0.7
                
        # Блок Завершенные НСТ
        if row['Тип сокр'] == 'Банкротство РБ':
                if row['Кол-во дней'] <= 60:
                    return 1.5
                if (row['Кол-во дней'] > 60) and (row['Кол-во дней'] <= 250):
                    return 1
                if (row['Кол-во дней'] > 250) and (row['Кол-во дней'] <= 370):
                    return 0.3
                if row['Кол-во дней'] > 370:
                    return 0.7
                
        if row['Тип сокр'] == 'Банкротство МБ':
            if row['Кол-во дней'] <= 80:
                return 1.5
            if (row['Кол-во дней'] > 80) and (row['Кол-во дней'] <= 390):
                return 1
            if (row['Кол-во дней'] > 390) and (row['Кол-во дней'] <= 510):
                return 0.3
            if row['Кол-во дней'] > 510:
                return 0.7

        if row['Тип сокр'] == 'Взыскание МБ':
            if row['Кол-во дней'] <= 180:
                return 1.5
            if (row['Кол-во дней'] > 180) and (row['Кол-во дней'] <= 300):
                return 1
            if (row['Кол-во дней'] > 300) and (row['Кол-во дней'] <= 420):
                return 0.3
            if row['Кол-во дней'] > 420:
                return 0.7

        if row['Тип сокр'] == 'Взыскание РБ':
            if row['Кол-во дней'] <= 160:
                return 1.5
            if (row['Кол-во дней'] > 160) and (row['Кол-во дней'] <= 270):
                return 1
            if (row['Кол-во дней'] > 270) and (row['Кол-во дней'] <= 390):
                return 0.3
            if row['Кол-во дней'] > 390:
                return 0.7
                
                
    # пошел блок активных дел (не завершенные)
    else:
        if row['СТ/НСТ'] == 'СТ':                       # активные СТ
            if row['Тип сокр'] == 'Банкротство РБ':
                if row['Кол-во дней'] < 100:
                    return 0
                if row['Кол-во дней'] > 100:
                    if (row['Кол-во дней'] >= 190) and (row['Кол-во дней'] <= 220):      
                        return 0
                    return 1
                    
                
            if row['Тип сокр'] == 'Банкротство МБ':
                if row['Кол-во дней'] < 170:
                    return 0
                if row['Кол-во дней'] > 170:
                    if (row['Кол-во дней'] >= 260) and (row['Кол-во дней'] <= 290):       
                        return 0
                    return 1
                
            if row['Тип сокр'] == 'Взыскание МБ':
                if row['Кол-во дней'] < 180:
                    return 0
                if row['Кол-во дней'] > 180:
                    if (row['Кол-во дней'] >= 270) and (row['Кол-во дней'] <= 300):       
                        return 0
                    return 1
                
            if row['Тип сокр'] == 'Взыскание РБ':
                if row['Кол-во дней'] < 170:
                    return 0
                if row['Кол-во дней'] > 170:
                    if (row['Кол-во дней'] >= 260) and (row['Кол-во дней'] <= 290):      
                        return 0
                    return 1
                
        # пошел блок активных НСТ
        if row['Тип сокр'] == 'Банкротство РБ':
                if row['Кол-во дней'] < 250:
                    return 0
                if row['Кол-во дней'] >= 250:
                    if (row['Кол-во дней'] >= 340) and (row['Кол-во дней'] <= 370):       
                        return 0
                    return 1   
                
        if row['Тип сокр'] == 'Банкротство МБ':
            if row['Кол-во дней'] < 390:
                return 0
            if row['Кол-во дней'] >= 390:
                if (row['Кол-во дней'] >= 480) and (row['Кол-во дней'] <= 510):       
                    return 0
                return 1

        if row['Тип сокр'] == 'Взыскание МБ':
            if row['Кол-во дней'] < 300:
                return 0
            if row['Кол-во дней'] >= 300:
                if (row['Кол-во дней'] >= 390) and (row['Кол-во дней'] <= 420):       
                    return 0
                return 1

        if row['Тип сокр'] == 'Взыскание РБ':
            if row['Кол-во дней'] < 270:
                return 0
            if row['Кол-во дней'] >= 270:
                if (row['Кол-во дней'] >= 360) and (row['Кол-во дней'] <= 390):      
                    return 0
                return 1
            
data_cuted['Коэф'] = data_cuted.apply(get_coef, axis=1)

__Помним что ранее учтенные дела не подлежат учету в СМОТ текущего квартала.__

Обнуляем коэф. по этим делам:

In [ ]:
data_cuted['Учтен'] = data_cuted['Учтен'].convert_dtypes()

In [ ]:
data_cuted['Учтен'].value_counts()

3 квартал 2021        7
4 квартал 2020        4
основное дело у ЮД    3
3 квартал 2019        2
2 квартал 2019        1
3 квартал 2022        1
4 квартал 2019        1
Name: Учтен, dtype: Int64

In [ ]:
# ф-ция обнуления коэф. по учтеным ранее делам
def check_early_doned(row):
    if type(row['Учтен']) == str:   # проверяю по типу данных, т.к. если не заполнено, то будет 0, а это цифра
        return 0
    return row['Коэф']
               
data_cuted['Коэф'] = data_cuted.apply(check_early_doned, axis=1)

In [ ]:
# проверям, должно быть 0 в коэф и наличие инф. об учете в поле Учтен
data_cuted.loc[data_cuted['Учтен'] == '3 квартал 2021']

,Номер по SIEBEL,РОО,Статус,Сумма заявленных требований,Сумма взыскания по решению суда,Дата создания,Дата конца,Комментарии,Дата закрытия,Учтен,Кол-во дней,Коэф,СТ/НСТ,Статус активности нст,Тип сокр,Общая сумма ПЗ по ОД,Сумма просроченных процентов,Факт,План
276,1-2921632422,Краснодар,В работе,8176283.11,1486753.05,2022-10-10,2023-03-31,"Подан иск по трем к/д, дело не назначено на 25...",NaT,3 квартал 2021,172,0.0,СТ,NaN,Взыскание МБ,0.00,0.0,None,None
305,1-2921632433,Краснодар,В работе,17888861.65,0.00,2022-10-10,2023-03-31,"Иск подан по трем КД, дело не назначено на 25....",NaT,3 квартал 2021,172,0.0,СТ,NaN,Взыскание МБ,0.00,0.0,None,None
306,1-2921632445,Краснодар,В работе,3675434.56,0.00,2022-10-10,2023-03-31,"Иск подан по трем КД, дело не назначено на 25....",NaT,3 квартал 2021,172,0.0,СТ,NaN,Взыскание МБ,0.00,0.0,None,None
839,1-2138028794,Краснодар,В работе,2247091.81,2247091.81,2021-12-28,2023-03-31,"иск направлен 10.01 с/з 09.02, 09.03 возражени...",2022-08-10,3 квартал 2021,458,0.0,НСТ,Завершено,Взыскание МБ,6270554.37,711908.4,None,None
840,1-2138028794,Краснодар,В работе,2247091.81,2247091.81,2021-12-28,2023-03-31,"иск направлен 10.01 с/з 09.02, 09.03 возражени...",2022-08-10,3 квартал 2021,458,0.0,НСТ,Завершено,Взыскание МБ,6270554.37,711908.4,None,None
841,1-2138028806,Краснодар,В работе,635509.83,635509.83,2021-12-28,2023-03-31,"иск направлен 10.01 с/з 09.02, 09.03 возражени...",2022-08-10,3 квартал 2021,458,0.0,НСТ,Завершено,Взыскание МБ,1520059.55,194033.9,None,None
842,1-2138028806,Краснодар,В работе,635509.83,635509.83,2021-12-28,2023-03-31,"иск направлен 10.01 с/з 09.02, 09.03 возражени...",2022-08-10,3 квартал 2021,458,0.0,НСТ,Завершено,Взыскание МБ,1520059.55,194033.9,None,None


In [ ]:
# в колонке Коэф. все корректно
data_cuted['Коэф'].value_counts()

0.0    641
1.0    359
1.5    154
0.3     50
0.7      1
Name: Коэф, dtype: int64

__Осталось занулить коэф по делам только об обращении взыскания на залог после ИНН:__

Это тоже особенность СМОТирования. <br>
В поле Комментарии содержатся в т.ч. Маркеры, которые заполняются ответственными, и по делам об обращении взыскания на залог после ИНН содержится маркер "обращение на залог после ИНН".

In [ ]:
data_cuted['Комментарии'] = data_cuted['Комментарии'].convert_dtypes()
data_cuted['Комментарии'][0:3]

0    Иск.Наследники. Иск направлен в суд - 19.10.20...
1    Иск. Наследники. Исковое заявление напр. в суд...
2    Иск.Наследники. Исковое заявление напр. в суд ...
Name: Комментарии, dtype: string

In [ ]:
# ищу вхождение части маркера в поле Комментарии
search_to = 'залог после'

indx_serched = data_cuted[data_cuted["Комментарии"].str.contains(search_to)].index

for indx in indx_serched:
    data_cuted.loc[indx, 'Коэф'] = 0

In [ ]:
data_cuted['Коэф'].value_counts()

0.0    685
1.0    332
1.5    139
0.3     48
0.7      1
Name: Коэф, dtype: int64

Видим изменения в кол-ве дел по коэффициентам.

Переходим в финальной части.

## Вычисляем факт и план:

In [ ]:
def compute_fact(row):
    if row['Статус'] == 'Завершено':
        sum_in_fact = row['Коэф'] * row['Сумма взыскания по решению суда']
        return sum_in_fact

def compute_plan(row):
    if row['Статус'] == 'Завершено':
        
        if (row['Коэф'] == 1) or(row['Коэф'] == 0.7):
            
            # по банкротам сразу заявл. требования будем тянуть
            if (row['Тип сокр'] == 'Банкротство РБ') or (row['Тип сокр'] == 'Банкротство МБ'):
                return row['Сумма заявленных требований']
            
            # по не банкротам суммируем ОД и %
            result = row['Общая сумма ПЗ по ОД'] + row['Сумма просроченных процентов']
            if result == 0:
                result = row['Сумма заявленных требований']
            
            return result
    
    # блок по не завершенным делам (активным)
    else:
        if row['Коэф'] == 1:
            if (row['Тип сокр'] == 'Банкротство РБ') or (row['Тип сокр'] == 'Банкротство МБ'):
                return row['Сумма заявленных требований']
            
            # по не банкротам
            result = row['Общая сумма ПЗ по ОД'] + row['Сумма просроченных процентов']
            if result == 0:
                result = row['Сумма заявленных требований']
            
            return result

data_cuted['План'] = data_cuted.apply(compute_plan, axis=1)
data_cuted['Факт'] = data_cuted.apply(compute_fact, axis=1)

In [ ]:
# РАЗОБРАТЬСЯ ПОЗЖЕ ОТКУДА ПОЯВЛЯЮТСЯ ДУБЛИКАТЫ 
data_cuted.drop_duplicates(keep='first',inplace=True)

## Формируем таблицу  result_table:

In [ ]:
plan = pd.pivot_table(data_cuted, values='План', index=['РОО'], columns=['Тип сокр'], aggfunc=np.sum)

fact = pd.pivot_table(data_cuted, values='Факт', index=['РОО'], columns=['Тип сокр'], aggfunc=np.sum)

In [ ]:
plan

Тип сокр,Банкротство МБ,Банкротство РБ,Взыскание МБ,Взыскание РБ
РОО,,,,
Волгоград,0.00,28366471.75,519371.15,4770735.62
Краснодар,97149862.23,53908823.25,6009620.63,16328494.07
Нальчик,1890182.18,21865480.47,NaN,4316070.26
Ростов,20841642.11,77662151.06,15105167.72,4201412.65
Ставрополь,NaN,5326736.08,3524417.50,1662527.41


In [ ]:
fact

Тип сокр,Банкротство МБ,Банкротство РБ,Взыскание МБ,Взыскание РБ
РОО,,,,
Волгоград,0.000,3.585302e+07,1903911.825,9355304.289
Краснодар,0.000,5.102533e+07,0.000,701116.536
Нальчик,0.000,2.984166e+07,NaN,3970965.267
Ростов,1976396.325,1.194121e+08,312285.345,0.000
Ставрополь,NaN,4.900104e+06,0.000,0.000


In [ ]:
for i in plan.columns:
    plan.rename(columns={i : i + ' план'}, inplace=True)

plan

Тип сокр,Банкротство МБ план,Банкротство РБ план,Взыскание МБ план,Взыскание РБ план
РОО,,,,
Волгоград,0.00,28366471.75,519371.15,4770735.62
Краснодар,97149862.23,53908823.25,6009620.63,16328494.07
Нальчик,1890182.18,21865480.47,NaN,4316070.26
Ростов,20841642.11,77662151.06,15105167.72,4201412.65
Ставрополь,NaN,5326736.08,3524417.50,1662527.41


In [ ]:
for i in fact.columns:
    fact.rename(columns={i : i + ' факт'}, inplace=True)

fact

Тип сокр,Банкротство МБ факт,Банкротство РБ факт,Взыскание МБ факт,Взыскание РБ факт
РОО,,,,
Волгоград,0.000,3.585302e+07,1903911.825,9355304.289
Краснодар,0.000,5.102533e+07,0.000,701116.536
Нальчик,0.000,2.984166e+07,NaN,3970965.267
Ростов,1976396.325,1.194121e+08,312285.345,0.000
Ставрополь,NaN,4.900104e+06,0.000,0.000


In [ ]:
fact = pd.merge(fact, plan,  on ='РОО', how ='left')
fact

Тип сокр,Банкротство МБ факт,Банкротство РБ факт,Взыскание МБ факт,Взыскание РБ факт,Банкротство МБ план,Банкротство РБ план,Взыскание МБ план,Взыскание РБ план
РОО,,,,,,,,
Волгоград,0.000,3.585302e+07,1903911.825,9355304.289,0.00,28366471.75,519371.15,4770735.62
Краснодар,0.000,5.102533e+07,0.000,701116.536,97149862.23,53908823.25,6009620.63,16328494.07
Нальчик,0.000,2.984166e+07,NaN,3970965.267,1890182.18,21865480.47,NaN,4316070.26
Ростов,1976396.325,1.194121e+08,312285.345,0.000,20841642.11,77662151.06,15105167.72,4201412.65
Ставрополь,NaN,4.900104e+06,0.000,0.000,NaN,5326736.08,3524417.50,1662527.41


In [ ]:
fact.fillna(0, inplace=True) 
fact

Тип сокр,Банкротство МБ факт,Банкротство РБ факт,Взыскание МБ факт,Взыскание РБ факт,Банкротство МБ план,Банкротство РБ план,Взыскание МБ план,Взыскание РБ план
РОО,,,,,,,,
Волгоград,0.000,3.585302e+07,1903911.825,9355304.289,0.00,28366471.75,519371.15,4770735.62
Краснодар,0.000,5.102533e+07,0.000,701116.536,97149862.23,53908823.25,6009620.63,16328494.07
Нальчик,0.000,2.984166e+07,0.000,3970965.267,1890182.18,21865480.47,0.00,4316070.26
Ростов,1976396.325,1.194121e+08,312285.345,0.000,20841642.11,77662151.06,15105167.72,4201412.65
Ставрополь,0.000,4.900104e+06,0.000,0.000,0.00,5326736.08,3524417.50,1662527.41


In [ ]:
fact.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, Волгоград to Ставрополь
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Банкротство МБ факт  5 non-null      float64
 1   Банкротство РБ факт  5 non-null      float64
 2   Взыскание МБ факт    5 non-null      float64
 3   Взыскание РБ факт    5 non-null      float64
 4   Банкротство МБ план  5 non-null      float64
 5   Банкротство РБ план  5 non-null      float64
 6   Взыскание МБ план    5 non-null      float64
 7   Взыскание РБ план    5 non-null      float64
dtypes: float64(8)
memory usage: 360.0+ bytes


In [ ]:
fact['% Банкротство РБ'] = fact['Банкротство РБ факт'] / fact['Банкротство РБ план'] * 100
fact['% Банкротство МБ'] = fact['Банкротство МБ факт'] / fact['Банкротство МБ план'] * 100
fact['% Взыскание РБ'] = fact['Взыскание РБ факт'] / fact['Взыскание РБ план'] * 100
fact['% Взыскание МБ'] = fact['Взыскание МБ факт'] / fact['Взыскание МБ план'] * 100
fact.fillna(0, inplace=True) 

result_table = fact[['Банкротство МБ факт', 'Банкротство МБ план', '% Банкротство МБ',\
              'Банкротство РБ факт', 'Банкротство РБ план', '% Банкротство РБ',\
              'Взыскание МБ факт', 'Взыскание МБ план', '% Взыскание МБ',\
              'Взыскание РБ факт', 'Взыскание РБ план', '% Взыскание РБ'
        ]]

perc_list = ['% Банкротство РБ','% Банкротство МБ',  '% Взыскание РБ', '% Взыскание МБ']

for i in perc_list:
    result_table[i] = result_table[i].apply(lambda x: float(f'{x: .2f}') )
    

result_table

Тип сокр,Банкротство МБ факт,Банкротство МБ план,% Банкротство МБ,Банкротство РБ факт,Банкротство РБ план,% Банкротство РБ,Взыскание МБ факт,Взыскание МБ план,% Взыскание МБ,Взыскание РБ факт,Взыскание РБ план,% Взыскание РБ
РОО,,,,,,,,,,,,
Волгоград,0.000,0.00,0.00,3.585302e+07,28366471.75,126.39,1903911.825,519371.15,366.58,9355304.289,4770735.62,196.10
Краснодар,0.000,97149862.23,0.00,5.102533e+07,53908823.25,94.65,0.000,6009620.63,0.00,701116.536,16328494.07,4.29
Нальчик,0.000,1890182.18,0.00,2.984166e+07,21865480.47,136.48,0.000,0.00,0.00,3970965.267,4316070.26,92.00
Ростов,1976396.325,20841642.11,9.48,1.194121e+08,77662151.06,153.76,312285.345,15105167.72,2.07,0.000,4201412.65,0.00
Ставрополь,0.000,0.00,0.00,4.900104e+06,5326736.08,91.99,0.000,3524417.50,0.00,0.000,1662527.41,0.00


__Сохраняю преобразованный датафрейм с эксель:__

In [ ]:
writer = pd.ExcelWriter('output.xlsx') # write dataframe to excel 
data_cuted.to_excel(writer) # save the excel 
writer.save()
